<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Unit-Test" data-toc-modified-id="Unit-Test-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Unit Test</a></span></li><li><span><a href="#Usage-Example" data-toc-modified-id="Usage-Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Usage Example</a></span><ul class="toc-item"><li><span><a href="#Define-Emission-Probability-Function" data-toc-modified-id="Define-Emission-Probability-Function-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define Emission Probability Function</a></span></li><li><span><a href="#Setting-up-an-HMM-Chain" data-toc-modified-id="Setting-up-an-HMM-Chain-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Setting up an HMM Chain</a></span></li><li><span><a href="#HMM-Optimisation-using-MCMC" data-toc-modified-id="HMM-Optimisation-using-MCMC-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>HMM Optimisation using MCMC</a></span></li></ul></li></ul></div>

# Introduction

This notebook provides insight into how to use the HMMLikelihood library. The library currently provides the capability to calculate the likelihood function of an HMM using OpenCL. 
The limitations and capability:
- Diagonal emission matrix, 
- Custom emission probability function.
- HMM limited to about 80 states.
- Developed for very long time sequences (> 10000).

The library origin is from work performed in *A 1000-fold Acceleration of Hidden Markov Model Fitting using Graphical Processing Units, with application to Nonvolcanic Tremor Classification* https://arxiv.org/abs/2003.03508

It is important to read the comments within each of the code blocks as well as the markdown text. Further in this chapter, all the libraries are imported and OpenCL devices interrogated.

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pylab as plt
import time
import timeit
import pandas as pd
import scipy.stats as sps

# MCMC import
import pytwalk

# Add HMMLikelihood path to python path
import sys
sys.path
sys.path.append('../../')

In [ ]:
# Add before importing HMMLikelihood to make opencl vebose
import os
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '0'
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import HMMLikelihood

In [ ]:
# Query OpenCL devices.
HMMLikelihood.Likelihood().pyOpenCLInfo()

Note. 

In function call `lhfunc = HMMLikelihood.Likelihood` 
the parameter `platform_id`can be changed to select 
the correct OpenCL platform. If unsure, default setting is 0.

`lhfunc.pyOpenCLInfo()` provide to info on OpenCL platforms.

# Unit Test

This section test the OpenCL device with accuracy tests and speed tests.

In [ ]:
# Number of data points
T = 1000
# Number of states
dim = 25
# Select the OpenCL platform, defautl is 0
platform_id = 0
# More output
beVerbose=False


# Files to opencl kernels
file_diag_kernel = "../../HMMLikelihood/prob_function.cl"
file_diag_normal = "../../HMMLikelihood/diag_normalc.cl"
file_diag_mat_mul = "../../HMMLikelihood/diag_mat_mulO.cl"
file_matrixmul = "../../HMMLikelihood/matrixmulT.cl"
#file_diag_normal = "./clLikelihood/old_diag_normal.cl"
#file_diag_mat_mul = "./clLikelihood/diag_mat_mulB.cl"
#file_matrixmul = "./clLikelihood/matrixmulT.cl"

# Calculate random transistion matrix
transistion_matrix = HMMLikelihood.calc_transition_matrix(dim)
if (beVerbose):
    print("Transistion Matrix\n", transistion_matrix)


#Generate random data with dimension of 2
data = np.random.random([T,3])
data[:,2] = np.round(data[:,2])

if (beVerbose):
    print("Data\n")#, data)

# Generate kernel with random initialisations
kernel = HMMLikelihood.Kernel(
           np.random.random([dim,2]), 
           np.random.random([dim,2]), 
           np.random.random(dim), 
           np.random.random(dim), 
           dim)

if (beVerbose):
    print("Kernels\n", kernel())

# Create likelhood function
lh_func = HMMLikelihood.Likelihood(
                        data,                                # Data points  
                        transistion_matrix,                  # Transistion matrix initial 
                        kernel(),                            # Kernel parameters
                        file_diag_kernel=file_diag_kernel,   # File to probability function
                        file_diag_normal=file_diag_normal,   # File to calculate probability function
                        file_diag_mat_mul=file_diag_mat_mul, # File for diagonal X matrix
                        file_matrixmul=file_matrixmul,       # File for matrix X matrix    
                        wrkUnit=64,                          # Can be tuned to gain better performance!
                        platform_id=platform_id              # Change if more than 1 OpenCL device is available
               )

# Print calculated stationary matrix
if (beVerbose):
    print("Stationary Matrix",  lh_func.transistion_matrix)

# Initialise GPU
lh_func.gpu_chain()

print("Calculating...")
# Calculate likelihood with CPU and GPU
l1 = 0
l1 = lh_func.cpu_calculation()
l2 = lh_func.gpu_calculation()
l3 = lh_func.forward()

# Print likelhood outputs.
print("CPU         ", l1)
print("GPU         ", l2)
print("Forward     ", l3)
print("CPUGPU diff ", l1-l2)
print("GPUFWD diff ", l2-l3)
print("CPUGPU perc ", np.abs((l2 - l1) / max(l1, l2))*100)
print("GPUFWD perc ", np.abs((l3 - l2) / max(l2, l3))*100)

HMMLikelihood.check_error(l1, l2)
 
# Example to update different parameters
for i in range(1):
    print(i,end=" ")
    
# Update data used in likelhood, but not changing amount
    data = np.random.random([T,3])
    data[:,2] = np.round(data[:,2])
    lh_func.gpu_update_data(data)
    l1 = lh_func.cpu_calculation()
    l2 = lh_func.gpu_calculation()
    if (beVerbose):
        print("Update Data")
        print(l1)
        print(l2)
    HMMLikelihood.check_error(l1, l2)

# Update transition matrix
    transistion_matrix = HMMLikelihood.calc_transition_matrix(dim)
    lh_func.gpu_update_transistion_matrix(transistion_matrix)
    l1 = lh_func.cpu_calculation()
    l2 = lh_func.gpu_calculation()
    if (beVerbose):
        print("Update Transistion Matrix")
        print(l1)
        print(l2)
    HMMLikelihood.check_error(l1, l2)

# Update Kernel parameters        
    kernel = HMMLikelihood.Kernel(
               np.random.random([dim,2]), 
               np.random.random([dim,2]), 
               np.random.random(dim), 
               np.random.random(dim), 
               dim)
    lh_func.gpu_update_kernels(kernel())
    l1 = lh_func.cpu_calculation()
    l2 = lh_func.gpu_calculation()
    if (beVerbose):
        print("Update Kernel")
        print(l1)
        print(l2)
    HMMLikelihood.check_error(l1, l2)
print("DONE.")

HMMLikelihood.plot_transition_matrix(lh_func.transistion_matrix)

In [ ]:
# Time each stage of the likelihood computation
lh_func.gpu_timing()

In [ ]:
# Get average calculation time for the likelihood using OpenCL
%timeit lh_func.gpu_calculation()

In [ ]:
# Get average computation time using forward algorithm
%timeit lh_func.forward()

In [ ]:
# Execute forwards algorithm with verbose, showing execution of kernel
lh_func.forward(True)

# Usage Example

## Define Emission Probability Function

!It is important to note that currently only a diagonal emission matrix is supported.!

We will now define the function used to calculate the emission probability given an observation(data) and the related parameters for the specific state(param).

The kernel_function takes two inputs, the function parameters `param` and the observation `data` for a single time instance. The HMMLikelihood library provide a function `testOpenCLFunction` which can be used to test the emission probability function before using it inside the HMM.

The `testOpenCLFunction` takes three inputs, the file name of the emission probability function, a list of parameter values and a set of observations for a single time instance. The emission probability function demonstrated here is a Gaussian function with two parameters, mean and sigma, using a single observation. The function is saved in a text file called `testfile.cl`.


In [ ]:
# The template required for the emission probability function
# Currently populated with a Gaussian function.
code = """
float kernel_function(float* param, float* data)  
{
    /* Add User Function Here */
    
    float f;
    float mu_x    = param[0];
    float sigma_x = param[1];

    float x = data[0];

    float b1 = pown( (x - mu_x) / sigma_x, 2  );
    float b2 = 1 / ( sigma_x * sqrt(2 * M_PI) );

    f = b2 * exp(-0.5 * b1);
    return f;
    /* Can only return a single value */
}
"""
# Write the opencl text file, this file can be edited directly with a text editor if required.
file = open("./testfile.cl","w")
file.write(code)
file.close()

In [ ]:
# Here we can test the Gaussian function with 
mean=0
sigma=1 
# with data point at 
dpoint = 0.1

param = [mean, sigma]
data  = [dpoint]
print(HMMLikelihood.testOpenCLFunction("./testfile.cl", param, data))

In [ ]:
# We can verify our Gaussian implementation using scipy function
import scipy
scipy.stats.norm.pdf(dpoint,mean,sigma)

In [ ]:
# It is observed that the accuracy of the scipy function and the OpenCL 
# function differ. The OpenCL function is limited to a float32. 
# We can demonstrate the loss in accuracy.

clf = HMMLikelihood.testOpenCLFunction("testfile.cl", param, data)[0]
scf = scipy.stats.norm.pdf(dpoint,mean,sigma)
print("Percentage loss: ", np.abs(clf - scf) / scf * 100)

## Setting up an HMM Chain

An HMM chain with 20 states and sequence length of 1000 is created. For this example the previous emission probability function is used. Just as an example the data, transition matrix and kernel parameters for each state is randomized.

The class is initialised using the `HMMLikelihood.Likelihood` and set to the variable `lh_func`. The forward algorithm using OpenCL can be executed by `lh_func.gpu_calculation()` providing the likelihood of the chain. The observational data can be updated with `lh_func.gpu_update_data(<data>)`. The transistion matrix can be updated through `lh_func.gpu_update_transistion_matrix(<transistion_matrix>)`. The kernel parameters can be updated through  `lh_func.gpu_update_kernels(<kernel>)`. It must be noted that changing either the number of states of time sequence length would require the initialisation of a new `HMMLikelihood.Likelihood` class.

In [ ]:
# General Testing Block
# Length of sequence in HMM chain
T = 1000
# Number of states
dim = 20
# Number of observations per time instance
dim_observations = 1
# Number of paramaters in Emission Probability Function
n_params = 2

# Files to opencl kernels
file_diag_kernel = "./testfile.cl"

# Calculate random transistion matrix
transistion_matrix = HMMLikelihood.calc_transition_matrix(dim)

#Generate random data with dimension of dim_observations
data = np.random.random([T,dim_observations])

# Generate kernel with random initialisations
# The Gaussian kernel uses two parameters
kernel = np.random.random((dim, n_params))

# Create likelhood function
lh_func = HMMLikelihood.Likelihood(
                        data,                                # Data points  
                        transistion_matrix,                  # Transistion matrix initial 
                        kernel,                              # Kernel parameters
                        file_diag_kernel=file_diag_kernel,   # File to probability function
                        wrkUnit=64,                         # Can be played around to increase speed
                        platform_id=0                        # OpenCL platform selection
               )

# Initialise GPU
lh_func.gpu_chain()

In [ ]:
# Execute OpenCL HMM chain forward algorithm
lh_func.gpu_calculation()

In [ ]:
%timeit lh_func.gpu_calculation()

## HMM Optimisation using MCMC

For more detail on the HMM and the MCMC see the paper *A 1000-fold Acceleration of Hidden Markov Model Fitting using Graphical Processing Units, with application to Nonvolcanic Tremor Classification* https://arxiv.org/abs/2003.03508

The data provided with the repository is generated shokoku region data and not the real data used within the paper.

In [ ]:
import pytwalk
###########################################################
# User Defined
###########################################################
# Select OpenCL Kernels
file_diag_kernel = "../../HMMLikelihood/prob_function.cl"
# Number of States in model 
dim = 25
# Limit number of timesteps(chain sequence length)
limit_chain = True
chain_limit = 1000

###########################################################
# Read Data
###########################################################
locations = pd.read_csv("../../data/shokoku_locations_fake.csv", index_col=False) #Geo location
df_locations = locations.drop(locations.columns[0], axis=1)

observations = pd.read_csv("../../data/shokoku_observations_fake.csv", index_col=False) #Binary indicator
df_observations = observations.drop(observations.columns[0], axis=1)

df_data = pd.concat([df_locations, df_observations],  axis=1)
data = df_data.values
data = np.ascontiguousarray(data)
update_data = data[0,0:2]
# normalise data
for i in range(data.shape[0]):
    if data[i,2]>0.5:
        update_data = data[i,0:2]
    if data[i,2]<0.5:
        data[i,0:2] = update_data

# Limits for Branch Support
data_min_0 = data[:,0].min()
data_max_0 = data[:,0].max()
data_min_1 = data[:,1].min()
data_max_1 = data[:,1].max()
        
# Use only first <chain_limit> timesteps of data
if (limit_chain):
    data = data[:chain_limit]



#Number of data points observed in time
T = data.shape[0]

print("[Dim: {}, T: {}, Kernel param:{}]".format(dim, T, 6))

# Generate random transistion matrix
transistion_matrix = HMMLikelihood.calc_transition_matrix(dim)
  
# Initialise kernel with initial values
# Use random kernel initial values
kernel = HMMLikelihood.Kernel(
           np.random.random([dim,2]), 
           np.random.random([dim,2]), 
           np.random.random(dim), 
           np.random.random(dim), 
           dim)

###########################################################
# Initialise Likelhood function
###########################################################
lh_func = HMMLikelihood.Likelihood(
                     data,                                # Data points  
                     transistion_matrix,                  # Transistion matrix initial
                     kernel(),                            # Kernel parameters
                     file_diag_kernel=file_diag_kernel,   # File to probability function
                     )
# Initialise GPU
lh_func.gpu_chain()

###########################################################
# Define MCMC
###########################################################
def log_prior(x):
    # Moet nog dink oor watse prior as daar min data is
    x =np.abs(x)
    temp = np.reshape(x[:dim**2], [dim,dim]) + 0.0001
    diagonal = 1 - ( np.sum(np.abs(temp), axis=1) - np.diag(np.abs(temp)))
    np.fill_diagonal(temp, diagonal)
    log_prior = 0
    for i in range(0,dim):
        log_prior += sps.dirichlet.logpdf(temp[i,:], alpha=0.99*np.ones(dim))
    return log_prior

def log_posterior(x):
    return -(logL(x)) #log_prior(x)) 

def logL(x):
    # Take first dim^2 parameters and form transition_matrix
    P = np.reshape(x[:dim**2], [dim,dim])
    diagonal = 1 - ( np.sum(np.abs(P), axis=1) - np.diag(np.abs(P)))
    np.fill_diagonal(P, diagonal)
    transistion_matrix = P
    
    # Take rest of parameters to define kernels
    x_sub = x[dim**2:]
    start_params_sub = start_params[dim**2:]
    kernel = HMMLikelihood.Kernel(np.reshape(x_sub[:2*dim],[dim,2]), np.reshape(x_sub[2*dim:4*dim],[dim,2]),
           x_sub[4*dim:5*dim], x_sub[5*dim:6*dim], dim)
    
    # Update Kernels and Transtion matrix
    lh_func.gpu_update_kernels(kernel()) 
    lh_func.gpu_update_transistion_matrix(transistion_matrix)

    # Calculate likelihodd
    test = lh_func.gpu_calculation()
 
    # Bottom limit, prefent NaN
    if np.isinf(test) or np.isnan(test):
        loglikelihood = -1000000
    else:
        loglikelihood = test
    return loglikelihood

def generate_params(dim):
    """
        Generate array of random parameters to be used within the MCMC
    """
    params = np.random.random(6*dim + dim**2)
    params[0:dim**2] = np.reshape(HMMLikelihood.calc_transition_matrix(dim), dim**2)
    index_1 = T*np.random.random(dim)
    index_2 = T*np.random.random(dim)
    params[dim**2:(dim**2 + 2*dim):2] = data[index_1.astype(int),0]
    params[(dim**2+1):(dim**2 + 2*dim):2] = data[index_2.astype(int),1]
    
    #sort the means
    x_sub = params[dim**2:dim**2+2*dim]
    mean = np.reshape(x_sub,[dim,2])
    dist = np.linalg.norm(mean, axis=1)
    params[dim**2:dim**2+2*dim] = np.ndarray.flatten(mean[np.argsort(dist)])
    return params

def gv(x,string):
    if string == "trans":
        y = x[0:dim**2]
    if string == "mean":
        y = x[dim**2:(dim**2+2*dim)]
    if string == "mean_0":
        y = x[dim**2:(dim**2+2*dim):2]
    if string == "mean_1":
        y = x[(dim**2+1):(dim**2+2*dim):2] 
    if string == "sig":
        y = x[dim**2+2*dim:(dim**2+4*dim)]
    if string == "rho":
        y = x[dim**2+4*dim:(dim**2+5*dim)]
    if string == "p":
        y = x[dim**2+5*dim:(dim**2+6*dim)]
    return y

def BranchSupp(x): 
    global data_min_0, data_max_0
    global data_min_1, data_max_1
    X = np.reshape(x[:dim**2], [dim,dim])
    # Find diagonal coefficients
    D = np.diag(np.abs(X)) 
    # Find row sum without diagonal
    S = np.sum(np.abs(X), axis=1) - D 
    
    x_sub = x[dim**2:]
    flag_trans = (all(0<gv(x,"trans")+0.0001)&all(gv(x,"trans")<=1)&all(1 > S))
    flag_mean_0 = (all(data_min_0<=gv(x,"mean_0"))&all(gv(x,"mean_0")<=data_max_0))
    flag_mean_1 = (all(data_min_1<=gv(x,"mean_1")+0.1)&all(gv(x,"mean_1")<=data_max_1+0.1))

    flag_sig = (all(0<=gv(x,"sig"))&all(gv(x,"sig")<=1))
    
    flag_rho = (all(-1<=gv(x,"rho"))&all(gv(x,"rho")<=1))
    
    flag_p = (all(0<=gv(x,"p"))&all(gv(x,"p")<=1))
    return (flag_trans&flag_mean_0&flag_mean_1&flag_sig&flag_rho&flag_p)

start_params = generate_params(dim)

#Starting parameters for MCMC chains
start_params_1 = start_params 
start_params_2 = start_params*0.999+0.000001 

#Initialize and run MCMC
Tree_Inference = pytwalk.pytwalk( n=6*dim + dim**2, U=log_posterior, Supp=BranchSupp)
Tree_Inference.Run(T=1000, x0=start_params_1, xp0=start_params_2 )
#########################################################################################

In [ ]:
# Print chain analysis
Tree_Inference.Ana()

In [ ]:
# Print transition matrix
x = Tree_Inference.Output
optimal_ind = np.argpartition(x[int(x.shape[0] * 0.2):,-1], -5)[-5:]
optimal_pos = np.argmin(x[int(x.shape[0] * 0.2):,-1])
P = np.reshape(x[optimal_pos,0:(dim)**2], [dim,dim])
HMMLikelihood.plot_transition_matrix(P)

In [ ]:
# Print distributions of parameters.
print("Likelihood")
x = Tree_Inference.Output
y = x[int(x.shape[0] * 0.2):,-1]
plt.hist(y[y<100000], bins = 100)
plt.show()
print("Parameters")
for i in range(dim):
    fig, ax = plt.subplots(2,3, figsize=(15,7))
    ax = ax.flatten()
    print("Kernel {}".format(i + 1))
    for j in range(6):
        ax[j].hist(x[int(x.shape[0]*0.2):,dim**2 + i*6 + j], bins = 100)
    plt.show()